Speech- CNN1

In [ ]:
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.io import wavfile
import plotly.graph_objects as go
import plotly
from plotly.subplots import make_subplots
import winsound
import time

from IAHOS import IAHOS
from extraction_performances import extraction_performances
from hyperparams_initialization import hyperparams_initialization
from plots import plot_IAHOS,plot_confusion_matrix
from plots import plot_training_accuracy,plot_validation_accuracy,plot_test_scores
from plots import plot_output_NN

import keras
from keras import backend as K
from keras.models import Sequential,Input,Model
from keras.layers import Conv2D,Conv1D,MaxPooling2D,AveragePooling1D,MaxPooling1D
from keras.layers import Dense,Flatten,Dropout
from keras import initializers,optimizers,backend as k
from keras_radam import RAdam
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.metrics import confusion_matrix,accuracy_score
import tensorflow as tf

In [ ]:
training_set=np.load('training_set.npy')
validation_set=np.load('validation_set.npy')
training_labels=np.load('training_labels.npy')
validation_labels=np.load('validation_labels.npy')
test_set=np.load('test_set.npy')
test_labels=np.load('test_labels.npy')
words_name=np.load('words_name.npy')
randomize = np.arange(len(training_set))
np.random.shuffle(randomize)
training_set = training_set[randomize]
training_labels = training_labels[randomize]
randomize = np.arange(len(validation_set))
np.random.shuffle(randomize)
validation_set = validation_set[randomize]
validation_labels = validation_labels[randomize]
percentage = 10
index1 = int(len(training_set)*percentage/100)
index2 = int(len(validation_set)*percentage/100)
training_set2 = training_set[0:index1].reshape((-1,40,98,1))
validation_set2 = validation_set[0:index2].reshape((-1,40,98,1))
training_set = training_set.reshape((-1,40,98,1))
validation_set = validation_set.reshape((-1,40,98,1))
training_labels2 = training_labels[0:index1]
validation_labels2 = validation_labels[0:index2]

In [ ]:
def Classifier(i,hm,optimizer,op):
    x = Input(shape=(40,98,1))
    x1 = (Conv2D(int(hm[0,i]),kernel_size=(40,int(hm[1,i])),activation='relu'))(x)
    x1 = Dropout(rate=hm[8,i])(x1)
    x1 = (MaxPooling2D(pool_size=(1,int(hm[2,i]))))(x1)
    for j in range(int(hm[3,i])):
        x1 = (Conv2D(int(hm[4,i]),kernel_size=(1,int(hm[5,i])),activation='relu'))(x1)
        x1 = Dropout(rate=hm[8,i])(x1)
        x1 = (MaxPooling2D(pool_size=(1,int(hm[6,i]))))(x1)
    y1 = Flatten()(x1)
    y2 = (Dense(int(hm[7,i]),activation='relu'))(y1)
    y2 = Dropout(rate=hm[8,i])(y2)
    y = (Dense(31,activation='softmax'))(y2)
    classifier=Model(inputs=x,outputs=y)
    if optimizer=='radam':
        Optimizer=RAdam()
    elif optimizer=='sgd':
        if op[3]==0:
            Optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
        else:
            Optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=True)
    elif optimizer=='rmsprop':
        Optimizer=keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
    elif optimizer=='adagrad':
        Optimizer=keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
    elif optimizer=='adadelta':
        Optimizer=keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
    elif optimizer=='adam':
        Optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    elif optimizer=='adamax':
        Optimizer=keras.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    else:
        Optimizer=keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    classifier.compile(optimizer=Optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    return classifier

In [ ]:
attempts=2
variables=9
iterations=attempts**variables
limits = [[4,64],[2,6],[1,2],[0,2],[2,64],[2,6],[1,2],[50,500],[0,0.5]]
method='grid'
rounds=2

In [ ]:
op = [[],[0.01,0,0,0],[0.001,0.9,0,0],[0.01,0,0],[1,0.95,0,0],[0.001,0.9,0.999,0,0,0],
      [0.002,0.9,0.999,0,0],[0.002,0.9,0.999,0,0.004]]
tgp,tgp2,ogp,ogp2,final=IAHOS(rounds,method,limits,attempts,variables,iterations,Classifier,
                       training_set,training_labels,validation_set,validation_labels)

In [ ]:
#hyperparameters studied for this neural network
y = ['#filter1','kernel1','maxpool1','#layers',
     '#filter2','kernel2','maxpool2','fully_dim','dropout']
plot_IAHOS(y,ogp,ogp2,tgp,tgp2)

In [ ]:
test_set = test_set.reshape(-1,40,98,1)
final = np.load('final.npy')
best_hp = np.zeros((variables,1))
for i in range(variables):
    best_hp[i]=final[i]

training_accuracy=[]
validation_accuracy=[]
test_scores=[]
optimizers = ['radam','sgd','rmsprop','adagrad','adadelta','adam','adamax','nadam']

epochs=30

j=0
for optimizer in tqdm(optimizers):
        mc = ModelCheckpoint('best_model_'+optimizer+'.h5', monitor='val_loss', mode='min',
                            verbose=0,save_best_only=True)
        classifier=Classifier(0,best_hp,optimizer,op[j])
        history=classifier.fit(training_set,training_labels,
                               validation_data=[validation_set,validation_labels],
                               epochs=epochs,batch_size=100,verbose=0,callbacks=[mc])
        training_accuracy.append(history.history['acc'])
        validation_accuracy.append(history.history['val_acc'])
        j+=1
        if j<7:
            K.clear_session()